In [224]:
import json
import pandas as pd
from datetime import datetime
from listas import priorfalls_list, delirium_list, dementia_list, depresyndr_list, uriincont_list, fecincont_list, pressulc_list, osteopor_list, sarcopenia_list, sleepdisturb_list, chrpain_list, iatrog_list, constipation_list, CVdisease_list, heartdisease_list, ND_list, DM_list, hepatopat_list, neopl_list, AcuteRenalF_list, dizsyn_list, VIH_list, psicosis_list

from auxiliary_functions import valores_codigos, contar_diccionarios, dias_ingreso_total, asignar_intervalo_edad, sumar_barthel, sumar_emina, obtener_ultimo_resultat, obtener_valor_promedio, canadenca_comparada, disfagia_mecvvs, extraer_valor_clave, extraer_valor_clave_simple

tabla = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency


In [225]:
tabla.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,albumina,proteinas totales,Hb,colesterol total,leucos,limfos,prot C react,urea,FGE MDRD,FGE CDK-EPI
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,None,None,None,None,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,3.1,5.3,14.5,124.7,X,38.9,19.61,25.7,>60,>90
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,None,None,None,None,None,None
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,X,None,X,X,0.54,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...


In [226]:
CON_PA = tabla[tabla["PA_diagnosticada"] == 1.0]
CON_PA_MECVV = tabla[(tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] < 30)]
SIN_PA = tabla[(tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] > 30) | (tabla["P_diagnosticada"] == 1.0)]

# Sindromes geriatricos

## Indice de Charslton

## Caidas previas

In [227]:
caprev_CON_PA= (CON_PA['caidas_previas'] == 1.0).mean() * 100
print(f"Porcentaje de caidas previas: {caprev_CON_PA:.2f}%")

Porcentaje de caidas previas: 5.88%


In [228]:
caprev_CON_PA_MECVV= (CON_PA_MECVV['caidas_previas'] == 1.0).mean() * 100
print(f"Porcentaje de caidas previas: {caprev_CON_PA_MECVV:.2f}%")

Porcentaje de caidas previas: 0.00%


In [229]:
caprev_SIN_PA= (SIN_PA['caidas_previas'] == 1.0).mean() * 100
print(f"Porcentaje de caidas previas: {caprev_SIN_PA:.2f}%")

Porcentaje de caidas previas: 6.12%


#### Pvalor

##### Normalidad

In [230]:
stat, p_value = shapiro(CON_PA['caidas_previas'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.2622
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [231]:
stat, p_value = shapiro(CON_PA_MECVV['caidas_previas'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [232]:
stat, p_value = shapiro(SIN_PA['DO_diagnosticada'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.4716
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [233]:
stat, p_value_12 = mannwhitneyu(CON_PA['caidas_previas'], CON_PA_MECVV['caidas_previas'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.49002535950179893


In [234]:
stat, p_value_13 = mannwhitneyu(CON_PA['caidas_previas'], SIN_PA['caidas_previas'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.9858477456716862


In [235]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['caidas_previas'], SIN_PA['caidas_previas'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.44143809205435813


## Delirios

In [236]:
del_CON_PA= (CON_PA['delirios'] == 1.0).mean() * 100
print(f"Porcentaje de delirios: {del_CON_PA:.2f}%")

Porcentaje de delirios: 23.53%


In [237]:
del_CON_PA_MECVV= (CON_PA_MECVV['delirios'] == 1.0).mean() * 100
print(f"Porcentaje de delirios: {del_CON_PA_MECVV:.2f}%")

Porcentaje de delirios: 40.00%


In [238]:
del_SIN_PA= (SIN_PA['delirios'] == 1.0).mean() * 100
print(f"Porcentaje de delirios: {del_SIN_PA:.2f}%")

Porcentaje de delirios: 18.37%


#### Pvalor

##### Normalidad

In [239]:
stat, p_value = shapiro(CON_PA['delirios'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.5327
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [240]:
stat, p_value = shapiro(CON_PA_MECVV['delirios'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.6405
Valor p = 0.0002
El número de filas no sigue una distribución normal (rechazar H0).


In [241]:
stat, p_value = shapiro(SIN_PA['delirios'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.4716
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [242]:
stat, p_value_12 = mannwhitneyu(CON_PA['delirios'], CON_PA_MECVV['delirios'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.39174655228054167


In [243]:
stat, p_value_13 = mannwhitneyu(CON_PA['delirios'], SIN_PA['delirios'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.6549000779898075


In [244]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['delirios'], SIN_PA['delirios'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.13960802912048415


## Demencia

In [245]:
dem_CON_PA= (CON_PA['demencia'] == 1.0).mean() * 100
print(f"Porcentaje de demencia: {dem_CON_PA:.2f}%")

Porcentaje de demencia: 47.06%


In [246]:
dem_CON_PA_MECVV= (CON_PA_MECVV['demencia'] == 1.0).mean() * 100
print(f"Porcentaje de demencia: {dem_CON_PA_MECVV:.2f}%")

Porcentaje de demencia: 40.00%


In [247]:
dem_SIN_PA= (SIN_PA['demencia'] == 1.0).mean() * 100
print(f"Porcentaje de demencia: {dem_SIN_PA:.2f}%")

Porcentaje de demencia: 30.61%


#### Pvalor

##### Normalidad

In [248]:
stat, p_value = shapiro(CON_PA['demencia'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.6418
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [249]:
stat, p_value = shapiro(CON_PA_MECVV['demencia'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.6405
Valor p = 0.0002
El número de filas no sigue una distribución normal (rechazar H0).


In [250]:
stat, p_value = shapiro(SIN_PA['delirios'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.4716
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [251]:
stat, p_value_12 = mannwhitneyu(CON_PA['demencia'], CON_PA_MECVV['demencia'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.7484826918143946


In [252]:
stat, p_value_13 = mannwhitneyu(CON_PA['demencia'], SIN_PA['demencia'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.22703869678020527


In [253]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['demencia'], SIN_PA['demencia'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.5743933608939569


## Sindrome depresivo 

In [254]:
depre_CON_PA= (CON_PA['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Porcentaje de sindrome depresivo: {depre_CON_PA:.2f}%")

Porcentaje de sindrome depresivo: 23.53%


In [255]:
depre_CON_PA_MECVV= (CON_PA_MECVV['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Porcentaje de sindrome depresivo: {depre_CON_PA_MECVV:.2f}%")

Porcentaje de sindrome depresivo: 0.00%


In [256]:
depre_SIN_PA= (SIN_PA['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Porcentaje de sindrome depresivo: {depre_SIN_PA:.2f}%")

Porcentaje de sindrome depresivo: 14.29%


#### Pvalor

##### Normalidad

In [257]:
stat, p_value = shapiro(CON_PA['sindrome_depresivo'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.5327
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [258]:
stat, p_value = shapiro(CON_PA_MECVV['sindrome_depresivo'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


In [259]:
stat, p_value = shapiro(SIN_PA['sindrome_depresivo'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.4168
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [260]:
stat, p_value_12 = mannwhitneyu(CON_PA['sindrome_depresivo'], CON_PA_MECVV['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.11180527770715322


In [261]:
stat, p_value_13 = mannwhitneyu(CON_PA['sindrome_depresivo'], SIN_PA['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.3880726918610017


In [262]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['sindrome_depresivo'], SIN_PA['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.21340049774345338


## Incontinencia urinaria

In [263]:
ic_CON_PA= (CON_PA['incont_uri'] == 1.0).mean() * 100
print(f"Porcentaje de incontinencia urinaria: {ic_CON_PA:.2f}%")

Porcentaje de incontinencia urinaria: 29.41%


In [264]:
ic_CON_PA_MECVV= (CON_PA_MECVV['incont_uri'] == 1.0).mean() * 100
print(f"Porcentaje de incontinencia urinaria: {ic_CON_PA_MECVV:.2f}%")

Porcentaje de incontinencia urinaria: 30.00%


In [265]:
ic_SIN_PA= (SIN_PA['incont_uri'] == 1.0).mean() * 100
print(f"Porcentaje de incontinencia urinaria: {ic_SIN_PA:.2f}%")

Porcentaje de incontinencia urinaria: 14.29%


#### Pvalor

##### Normalidad

In [266]:
stat, p_value = shapiro(CON_PA['incont_uri'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.5785
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [267]:
stat, p_value = shapiro(CON_PA_MECVV['incont_uri'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.5942
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [268]:
stat, p_value = shapiro(SIN_PA['incont_uri'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.4168
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [269]:
stat, p_value_12 = mannwhitneyu(CON_PA['incont_uri'], CON_PA_MECVV['incont_uri'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 1.0


In [270]:
stat, p_value_13 = mannwhitneyu(CON_PA['incont_uri'], SIN_PA['incont_uri'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.17015925929193942


In [271]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['incont_uri'], SIN_PA['incont_uri'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.237514563226218


## Incontinencia fecal 

In [272]:
ifec_CON_PA= (CON_PA['incont_fec'] == 1.0).mean() * 100
print(f"Porcentaje de incontinencia fecal: {ifec_CON_PA:.2f}%")

Porcentaje de incontinencia fecal: 29.41%


In [273]:
ifec_CON_PA_MECVV= (CON_PA_MECVV['incont_fec'] == 1.0).mean() * 100
print(f"Porcentaje de incontinencia fecal: {ifec_CON_PA_MECVV:.2f}%")

Porcentaje de incontinencia fecal: 10.00%


In [274]:
ifec_SIN_PA= (SIN_PA['incont_fec'] == 1.0).mean() * 100
print(f"Porcentaje de incontinencia fecal: {ifec_SIN_PA:.2f}%")

Porcentaje de incontinencia fecal: 12.24%


#### Pvalor

##### Normalidad

In [275]:
stat, p_value = shapiro(CON_PA['incont_fec'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.5785
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [276]:
stat, p_value = shapiro(CON_PA_MECVV['incont_fec'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.3657
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [277]:
stat, p_value = shapiro(SIN_PA['incont_fec'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.3844
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [278]:
stat, p_value_12 = mannwhitneyu(CON_PA['incont_fec'], CON_PA_MECVV['incont_fec'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.2649031747093591


In [279]:
stat, p_value_13 = mannwhitneyu(CON_PA['incont_fec'], SIN_PA['incont_fec'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.10681676423525503


In [280]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['incont_fec'], SIN_PA['incont_fec'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.8568943909879498


## Ulceras de presion

In [281]:
up_CON_PA= (CON_PA['ulceras_presion'] == 1.0).mean() * 100
print(f"Porcentaje de ulceras de presion: {up_CON_PA:.2f}%")

Porcentaje de ulceras de presion: 5.88%


In [282]:
up_CON_PA_MECVV= (CON_PA_MECVV['ulceras_presion'] == 1.0).mean() * 100
print(f"Porcentaje de ulceras de presion: {up_CON_PA_MECVV:.2f}%")

Porcentaje de ulceras de presion: 0.00%


In [283]:
up_SIN_PA= (SIN_PA['ulceras_presion'] == 1.0).mean() * 100
print(f"Porcentaje de ulceras de presion: {up_SIN_PA:.2f}%")

Porcentaje de ulceras de presion: 2.04%


#### Pvalor

##### Normalidad

In [284]:
stat, p_value = shapiro(CON_PA['ulceras_presion'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.2622
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [285]:
stat, p_value = shapiro(CON_PA_MECVV['ulceras_presion'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


In [286]:
stat, p_value = shapiro(SIN_PA['ulceras_presion'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.1273
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [287]:
stat, p_value_12 = mannwhitneyu(CON_PA['ulceras_presion'], CON_PA_MECVV['ulceras_presion'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.49002535950179893


In [288]:
stat, p_value_13 = mannwhitneyu(CON_PA['ulceras_presion'], SIN_PA['ulceras_presion'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.4440308448923743


In [289]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['ulceras_presion'], SIN_PA['ulceras_presion'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.6843175494727258


## Immobilidad

## Confusion

## Osteoporosis

In [290]:
ost_CON_PA= (CON_PA['osteoporosis'] == 1.0).mean() * 100
print(f"Porcentaje de osteoporosis: {ost_CON_PA:.2f}%")

Porcentaje de osteoporosis: 0.00%


In [291]:
ost_CON_PA_MECVV= (CON_PA_MECVV['osteoporosis'] == 1.0).mean() * 100
print(f"Porcentaje de osteoporosis: {ost_CON_PA_MECVV:.2f}%")

Porcentaje de osteoporosis: 0.00%


In [292]:
ost_SIN_PA= (SIN_PA['osteoporosis'] == 1.0).mean() * 100
print(f"Porcentaje de osteoporosis: {ost_SIN_PA:.2f}%")

Porcentaje de osteoporosis: 0.00%


#### Pvalor

##### Normalidad

In [293]:
stat, p_value = shapiro(CON_PA['osteoporosis'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


In [294]:
stat, p_value_12 = mannwhitneyu(CON_PA['osteoporosis'], CON_PA_MECVV['osteoporosis'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 1.0


In [295]:
stat, p_value_12 = ttest_ind(CON_PA['osteoporosis'], CON_PA_MECVV['osteoporosis'], equal_var= False)
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV nan


## Sarcopenia

In [296]:
sarc_CON_PA= (CON_PA['sarcopenia'] == 1.0).mean() * 100
print(f"Porcentaje de sarcopenia: {sarc_CON_PA:.2f}%")

Porcentaje de sarcopenia: 0.00%


In [297]:
sarc_CON_PA_MECVV= (CON_PA_MECVV['sarcopenia'] == 1.0).mean() * 100
print(f"Porcentaje de sarcopenia: {sarc_CON_PA_MECVV:.2f}%")

Porcentaje de sarcopenia: 10.00%


In [298]:
sarc_SIN_PA= (SIN_PA['sarcopenia'] == 1.0).mean() * 100
print(f"Porcentaje de sarcopenia: {sarc_SIN_PA:.2f}%")

Porcentaje de sarcopenia: 4.08%


#### Pvalor

##### Normalidad

In [299]:
stat, p_value = shapiro(CON_PA['sarcopenia'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


In [300]:
stat, p_value = shapiro(CON_PA_MECVV['sarcopenia'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.3657
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [301]:
stat, p_value = shapiro(SIN_PA['sarcopenia'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.2008
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [302]:
stat, p_value_12 = mannwhitneyu(CON_PA['sarcopenia'], CON_PA_MECVV['sarcopenia'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.21976847950216105


In [303]:
stat, p_value_13 = mannwhitneyu(CON_PA['sarcopenia'], SIN_PA['sarcopenia'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.41519527301502746


In [304]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['sarcopenia'], SIN_PA['sarcopenia'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.45734646700671056


## Trastorno del sueño

In [305]:
ts_CON_PA= (CON_PA['problsueño'] == 1.0).mean() * 100
print(f"Porcentaje de trastornos del sueño: {ts_CON_PA:.2f}%")

Porcentaje de trastornos del sueño: 0.00%


In [306]:
ts_CON_PA_MECVV= (CON_PA_MECVV['problsueño'] == 1.0).mean() * 100
print(f"Porcentaje de trastornos del sueño: {ts_CON_PA_MECVV:.2f}%")

Porcentaje de trastornos del sueño: 0.00%


In [307]:
ts_SIN_PA= (SIN_PA['problsueño'] == 1.0).mean() * 100
print(f"Porcentaje de trastornos del sueño: {ts_SIN_PA:.2f}%")

Porcentaje de trastornos del sueño: 0.00%


## Dolor cronico

In [308]:
dc_CON_PA= (CON_PA['dolor_cron'] == 1.0).mean() * 100
print(f"Porcentaje de dolor cronico: {dc_CON_PA:.2f}%")

Porcentaje de dolor cronico: 0.00%


In [309]:
dc_CON_PA_MECVV= (CON_PA_MECVV['dolor_cron'] == 1.0).mean() * 100
print(f"Porcentaje de dolor cronico: {dc_CON_PA_MECVV:.2f}%")

Porcentaje de dolor cronico: 0.00%


In [310]:
dc_SIN_PA= (SIN_PA['dolor_cron'] == 1.0).mean() * 100
print(f"Porcentaje de dolor cronico: {dc_SIN_PA:.2f}%")

Porcentaje de dolor cronico: 0.00%


## Iatrogenico

In [311]:
iatr_CON_PA= (CON_PA['iatrogenico'] == 1.0).mean() * 100
print(f"Porcentaje de iatrogenico: {iatr_CON_PA:.2f}%")

Porcentaje de iatrogenico: 0.00%


In [312]:
iatr_CON_PA_MECVV= (CON_PA_MECVV['iatrogenico'] == 1.0).mean() * 100
print(f"Porcentaje de iatrogenico: {iatr_CON_PA_MECVV:.2f}%")

Porcentaje de iatrogenico: 0.00%


In [313]:
iatr_SIN_PA= (SIN_PA['iatrogenico'] == 1.0).mean() * 100
print(f"Porcentaje de iatrogenico: {iatr_SIN_PA:.2f}%")

Porcentaje de iatrogenico: 0.00%


## Estreñimiento

In [314]:
est_CON_PA= (CON_PA['estreñimiento'] == 1.0).mean() * 100
print(f"Porcentaje de estreñimiento: {est_CON_PA:.2f}%")

Porcentaje de estreñimiento: 0.00%


In [315]:
est_CON_PA_MECVV= (CON_PA_MECVV['estreñimiento'] == 1.0).mean() * 100
print(f"Porcentaje de estreñimiento: {est_CON_PA_MECVV:.2f}%")

Porcentaje de estreñimiento: 0.00%


In [316]:
est_SIN_PA= (SIN_PA['estreñimiento'] == 1.0).mean() * 100
print(f"Porcentaje de estreñimiento: {est_SIN_PA:.2f}%")

Porcentaje de estreñimiento: 0.00%


# Historial medico/comorbilidades

## Enfermedades CV

In [317]:
cv_CON_PA= (CON_PA['CV'] == 1.0).mean() * 100
print(f"Porcentaje de problemas CV: {cv_CON_PA:.2f}%")

Porcentaje de problemas CV: 5.88%


In [318]:
cv_CON_PA_MECVV= (CON_PA_MECVV['CV'] == 1.0).mean() * 100
print(f"Porcentaje de problemas CV: {cv_CON_PA_MECVV:.2f}%")

Porcentaje de problemas CV: 10.00%


In [319]:
cv_SIN_PA= (SIN_PA['CV'] == 1.0).mean() * 100
print(f"Porcentaje de problemas CV: {cv_SIN_PA:.2f}%")

Porcentaje de problemas CV: 6.12%


#### Pvalor

##### Normalidad 

In [320]:
stat, p_value = shapiro(CON_PA['CV'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.2622
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [321]:
stat, p_value = shapiro(CON_PA_MECVV['CV'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.3657
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [322]:
stat, p_value = shapiro(SIN_PA['CV'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.2580
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [323]:
stat, p_value_12 = mannwhitneyu(CON_PA['CV'], CON_PA_MECVV['CV'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.7400102360583588


In [324]:
stat, p_value_13 = mannwhitneyu(CON_PA['CV'], SIN_PA['CV'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.9858477456716862


In [325]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['CV'], SIN_PA['CV'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.6762982127217047


## Enfermedades del corazon

In [327]:
cora_CON_PA= (CON_PA['probl_corazon'] == 1.0).mean() * 100
print(f"Porcentaje de enfermedades del corazon: {cora_CON_PA:.2f}%")

Porcentaje de enfermedades del corazon: 11.76%


In [328]:
cora_CON_PA_MECVV= (CON_PA_MECVV['probl_corazon'] == 1.0).mean() * 100
print(f"Porcentaje de enfermedades del corazon: {cora_CON_PA_MECVV:.2f}%")

Porcentaje de enfermedades del corazon: 10.00%


In [329]:
cora_SIN_PA= (SIN_PA['probl_corazon'] == 1.0).mean() * 100
print(f"Porcentaje de enfermedades del corazon: {cora_SIN_PA:.2f}%")

Porcentaje de enfermedades del corazon: 14.29%


#### Pvalor

##### Normalidad

In [332]:
stat, p_value = shapiro(CON_PA['probl_corazon'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.3853
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [333]:
stat, p_value = shapiro(CON_PA_MECVV['probl_corazon'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.3657
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [334]:
stat, p_value = shapiro(SIN_PA['probl_corazon'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.4168
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [335]:
stat, p_value_12 = mannwhitneyu(CON_PA['probl_corazon'], CON_PA_MECVV['probl_corazon'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.9265568396984611


In [336]:
stat, p_value_13 = mannwhitneyu(CON_PA['CV'], SIN_PA['CV'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.9858477456716862


In [337]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['CV'], SIN_PA['CV'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.6762982127217047


## Enfermedades neurodegenerativas

In [340]:
nd_CON_PA= (CON_PA['neurodegenerativas'] == 1.0).mean() * 100
print(f"Porcentaje de enfermedades neurodegenerativas: {nd_CON_PA:.2f}%")

Porcentaje de enfermedades neurodegenerativas: 0.00%


In [341]:
nd_CON_PA_MECVV= (CON_PA_MECVV['neurodegenerativas'] == 1.0).mean() * 100
print(f"Porcentaje de enfermedades neurodegenerativas: {nd_CON_PA_MECVV:.2f}%")

Porcentaje de enfermedades neurodegenerativas: 10.00%


In [342]:
nd_SIN_PA= (SIN_PA['neurodegenerativas'] == 1.0).mean() * 100
print(f"Porcentaje de enfermedades neurodegenerativas: {nd_SIN_PA:.2f}%")

Porcentaje de enfermedades neurodegenerativas: 4.08%


#### Pvalor

##### Normalidad

In [345]:
stat, p_value = shapiro(CON_PA['neurodegenerativas'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [346]:
stat, p_value = shapiro(CON_PA_MECVV['neurodegenerativas'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.3657
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [347]:
stat, p_value = shapiro(SIN_PA['neurodegenerativas'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.2008
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [348]:
stat, p_value_12 = mannwhitneyu(CON_PA['neurodegenerativas'], CON_PA_MECVV['neurodegenerativas'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.21976847950216105


In [349]:
stat, p_value_13 = mannwhitneyu(CON_PA['neurodegenerativas'], SIN_PA['neurodegenerativas'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.41519527301502746


In [350]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['neurodegenerativas'], SIN_PA['neurodegenerativas'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.45734646700671056


## Diabetes mellitus

In [352]:
dm_CON_PA= (CON_PA['DM'] == 1.0).mean() * 100
print(f"Porcentaje de diabetes mellitus: {dm_CON_PA:.2f}%")

Porcentaje de diabetes mellitus: 23.53%


In [353]:
dm_CON_PA_MECVV= (CON_PA_MECVV['DM'] == 1.0).mean() * 100
print(f"Porcentaje de diabetes mellitus: {dm_CON_PA_MECVV:.2f}%")

Porcentaje de diabetes mellitus: 10.00%


In [354]:
dm_SIN_PA= (SIN_PA['DM'] == 1.0).mean() * 100
print(f"Porcentaje de diabetes mellitus: {dm_SIN_PA:.2f}%")

Porcentaje de diabetes mellitus: 26.53%


#### Pvalor 

##### Normalidad

In [357]:
stat, p_value = shapiro(CON_PA['DM'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.5327
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [358]:
stat, p_value = shapiro(CON_PA_MECVV['DM'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.3657
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [359]:
stat, p_value = shapiro(SIN_PA['DM'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.5514
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [360]:
stat, p_value_12 = mannwhitneyu(CON_PA['DM'], CON_PA_MECVV['DM'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.4120280546315189


In [361]:
stat, p_value_13 = mannwhitneyu(CON_PA['DM'], SIN_PA['DM'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PAN", p_value_13)

P-valor entre CON_PA y SIN_PAN 0.8163159339942008


In [362]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['DM'], SIN_PA['DM'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PAN", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PAN 0.2728301710651724


## Hepatopatias

In [363]:
hp_CON_PA= (CON_PA['hepatopatias'] == 1.0).mean() * 100
print(f"Porcentaje de hepatopatias: {hp_CON_PA:.2f}%")

Porcentaje de hepatopatias: 11.76%


In [364]:
hp_CON_PA_MECVV= (CON_PA_MECVV['hepatopatias'] == 1.0).mean() * 100
print(f"Porcentaje de hepatopatias: {hp_CON_PA_MECVV:.2f}%")

Porcentaje de hepatopatias: 10.00%


In [365]:
hp_SIN_PA= (SIN_PA['hepatopatias'] == 1.0).mean() * 100
print(f"Porcentaje de hepatopatias: {hp_SIN_PA:.2f}%")

Porcentaje de hepatopatias: 6.12%


#### Pvalor

##### Normalidad

In [368]:
stat, p_value = shapiro(CON_PA['hepatopatias'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.3853
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [369]:
stat, p_value = shapiro(CON_PA_MECVV['hepatopatias'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.3657
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [370]:
stat, p_value = shapiro(SIN_PA['hepatopatias'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.2580
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [371]:
stat, p_value_12 = mannwhitneyu(CON_PA['hepatopatias'], CON_PA_MECVV['hepatopatias'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.9265568396984611


In [375]:
stat, p_value_13 = mannwhitneyu(CON_PA['hepatopatias'], SIN_PA['hepatopatias'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.4618706673466527


In [377]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['hepatopatias'], SIN_PA['hepatopatias'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.6762982127217047


## Neoplasia

In [378]:
neopl_CON_PA= (CON_PA['neoplasias'] == 1.0).mean() * 100
print(f"Porcentaje de neoplasia: {neopl_CON_PA:.2f}%")

Porcentaje de neoplasia: 0.00%


In [379]:
neopl_CON_PA_MECVV= (CON_PA_MECVV['neoplasias'] == 1.0).mean() * 100
print(f"Porcentaje de neoplasia: {neopl_CON_PA_MECVV:.2f}%")

Porcentaje de neoplasia: 0.00%


In [382]:
neopl_SIN_PA= (SIN_PA['neoplasias'] == 1.0).mean() * 100
print(f"Porcentaje de neoplasia: {neopl_SIN_PA:.2f}%")

Porcentaje de neoplasia: 6.12%


#### Pvalor

##### Normalidad

In [433]:
stat, p_value = shapiro(CON_PA['neoplasias'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [387]:
stat, p_value = shapiro(CON_PA_MECVV['hepatopatias'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.3657
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [388]:
stat, p_value = shapiro(SIN_PA['hepatopatias'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.2580
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [389]:
stat, p_value_12 = mannwhitneyu(CON_PA['neoplasias'], CON_PA_MECVV['neoplasias'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 1.0


In [390]:
stat, p_value_13 = mannwhitneyu(CON_PA['neoplasias'], SIN_PA['neoplasias'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_12)

P-valor entre CON_PA y SIN_PA 1.0


In [391]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['neoplasias'], SIN_PA['neoplasias'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.44143809205435813


## Enfermedad cronica renal (creatinina > 1.5mg/dL)

Dado que esta columna es de tipo object primero la pasaré a tipo numerico para poder operar con ella. 

In [504]:
# Convertir la columna 'creatinina' a numérica en ambos DataFrames
CON_PA.loc[:, 'creatinina'] = pd.to_numeric(CON_PA['creatinina'], errors='coerce')
CON_PA_MECVV.loc[:, 'creatinina'] = pd.to_numeric(CON_PA_MECVV['creatinina'], errors='coerce')
SIN_PA.loc[:, 'creatinina'] = pd.to_numeric(SIN_PA['creatinina'], errors='coerce')

In [479]:
ecr_CON_PA= (CON_PA['creatinina'] >= 1.5).mean() * 100
print(f"Porcentaje de enfermedad cronica renal con creatinina >1.5: {ecr_CON_PA:.2f}%")

Porcentaje de enfermedad cronica renal con creatinina >1.5: 0.00%


In [478]:
ecr_CON_PA_MECVV= (CON_PA_MECVV['creatinina'] > 1.5).mean() * 100
print(f"Porcentaje de enfermedad cronica renal con creatinina >1.5: {ecr_CON_PA_MECVV:.2f}%")

Porcentaje de enfermedad cronica renal con creatinina >1.5: 0.00%


In [503]:
ecr_SIN_PA= (SIN_PA['creatinina'] > 1.5).mean() * 100
print(f"Porcentaje de enfermedad cronica renal con creatinina >1.5: {ecr_SIN_PA:.2f}%")

Porcentaje de enfermedad cronica renal con creatinina >1.5: 6.00%


#### Pvalor

##### Normalidad

In [505]:
stat, p_value = shapiro(CON_PA['creatinina'].dropna())
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9549
Valor p = 0.7439
El número de filas sigue una distribución normal (aceptar H0).


In [506]:
stat, p_value = shapiro(CON_PA_MECVV['creatinina'].dropna())
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9815
Valor p = 0.9110
El número de filas sigue una distribución normal (aceptar H0).


In [507]:
stat, p_value = shapiro(SIN_PA['creatinina'].dropna())
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.7235
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [508]:
stat, p_value_12 = ttest_ind(CON_PA['creatinina'], CON_PA_MECVV['creatinina'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV nan


In [517]:
stat, p_value_13 = mannwhitneyu(CON_PA['creatinina'], SIN_PA['creatinina'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA nan


In [516]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['creatinina'], SIN_PA['creatinina'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA nan
